In [12]:
import dpkt
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
from Utils import *
from DDBSCAN import Raster_DBSCAN
import open3d as op3
from open3d.web_visualizer import draw
from VisulizerTools import *
import time
from sklearn.cluster import DBSCAN
import cv2 as cv

In [9]:
file_path = r'D:\LiDAR_Data\MidTown\Thoma\2021-12-18-12-0-0.pcap'
loader = TDmapLoader(file_path).frame_gen()

In [10]:
Td_maps = []
for td in tqdm(loader):
    if td is not None:
        Td_maps.append(td)
    else:
        break

17964it [10:42, 27.98it/s]


In [13]:
def get_maps(db,Td_map,threshold_map):
    Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
    Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
    return Foreground_map,Labeling_map

In [ ]:
flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None,0.5, 3, 15, 3, 5, 1.2, 0)

In [163]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   
Td_map = Td_maps[-1]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_1')

In [189]:
clustering_td = Raster_DBSCAN(window_size=(5,15),eps = 1.2, min_samples= 12, Td_map_szie=threshold_map.shape)   
Td_map = Td_maps[20]
Foreground_map = (Td_map < threshold_map)&(Td_map != 0)
Labeling_map = clustering_td.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= threshold_map)&(Td_map != 0)
pcd = get_pcd_colored_filtered(Td_map,Labeling_map,Foreground_map)
draw(pcd,height=1000,width=1000)

WebVisualizer(window_uid='window_3')

In [180]:
Foreground_map = np.full(Td_maps[0].shape,True)
Labeling_map = np.ones_like(Foreground_map)
pcd = get_pcd_colored(Td_maps[0],Labeling_map)

In [181]:
draw(pcd)

WebVisualizer(window_uid='window_0')